In [2]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test_with_solutions.csv')

In [3]:
#make sure that count(insult) == count(non_insult) in train data
X_train_ins = train_data.loc[train_data['Insult'] == 1]['Comment'].values
y_train_ins = train_data.loc[train_data['Insult'] == 1]['Insult'].values

X_train_non_ins = train_data.loc[train_data['Insult'] == 0]['Comment'].values
y_train_non_ins = train_data.loc[train_data['Insult'] == 0]['Insult'].values

X_train = []
y_train = []
for i in range(len(X_train_ins)):
    X_train.append(X_train_ins[i])
    X_train.append(X_train_non_ins[i])
    
    y_train.append(y_train_ins[i])
    y_train.append(y_train_non_ins[i])


X_test = test_data['Comment'].values
y_test = test_data['Insult'].values

In [4]:
import re

def get_words(line):
    return [x for x in re.split('[_\n ,.?!@#$%^&*():;"\'\/\\0-9\{2}\\\]', line.lower().strip()) if x != '']

def append_words_from_lines(words, lines):
    for line in lines:
        for word in get_words(line):
            if word not in words and word:
                words.append(word)

vocab = []

append_words_from_lines(vocab, train_data['Comment'].values)
append_words_from_lines(vocab, test_data['Comment'].values)

In [5]:
len(vocab)

21289

In [6]:
print(vocab[:300])

['you', 'fuck', 'your', 'dad', 'i', 'really', 'don', 't', 'understand', 'point', 'xa', 'it', 'seems', 'that', 'are', 'mixing', 'apples', 'and', 'oranges', 'a', 'xc', 'majority', 'of', 'canadians', 'can', 'has', 'been', 'wrong', 'before', 'now', 'will', 'be', 'again', 'n', 'nunless', 're', 'supportive', 'the', 'idea', 'nothing', 'is', 'full', 'proof', 'or', 'perfect', 'so', 'take', 'chances', 'if', 'we', 'should', 'inadvertently', 'kill', 'son', 'daughter', 'then', 'them', 's', 'breaks', 'always', 'regard', 'as', 'collateral', 'damage', 'like', 'in', 'wartime', 'sorry', 'but', 'cheques', 'mail', 'listen', 'dont', 'wanna', 'get', 'married', 'to', 'man', 'women', 'do', 'what', 'would', 'bother', 'gay', 'people', 'got', 'stay', 'lane', 'let', 'god', 'nice', 'quick', 'judg', 'not', 'him', 'thought', 'wasnt', 'suppose', 'judge', 'c', 'xe', 'b', 'u', 'ea', 'xu', 'ed', 'ng', 'eddng', 'bi', 'ec', 'xecnh', 'xf', 'ho', 'kh', 'nc', 'd', 'cu', 'xed', 'chi', 'ee', 'nh', 'gi', 'eef', 'edf', 'v', 'gia

In [7]:
import word2vecReader
import numpy as np

w2v = word2vecReader.Word2Vec.load_word2vec_format('word2vec_twitter_model/word2vec_twitter_model.bin', binary=True)

In [8]:
word_to_vec = {}
for word in vocab:
    if w2v.__contains__(word):
        word_to_vec[word] = w2v[word]
    else:
        word_to_vec[word] = np.zeros(400)
    

In [9]:
del w2v

In [10]:
import torch

seq_len = 20


X_train_vec_seq = []
for i, line in enumerate(X_train[::]):
    X_train_vec_seq.append([])
    words = get_words(line)
    for j in range(seq_len):
        if j < len(words):
            X_train_vec_seq[-1].append(word_to_vec[words[j]])
        else:
            X_train_vec_seq[-1].append(np.zeros(400))
X_train_vec_seq = torch.from_numpy(np.array(X_train_vec_seq)).type(torch.FloatTensor)
            
X_test_vec_seq = []
for i, line in enumerate(X_test[::]):
    X_test_vec_seq.append([])
    words = get_words(line)
    for j in range(seq_len):
        if j < len(words):
            X_test_vec_seq[-1].append(word_to_vec[words[j]])
        else:
            X_test_vec_seq[-1].append(np.zeros(400))
X_test_vec_seq = torch.from_numpy(np.array(X_test_vec_seq)).type(torch.FloatTensor)

y_train_torch = torch.from_numpy(np.array(y_train[::]))   
y_test_torch = torch.from_numpy(np.array(test_data['Insult'].values))

In [11]:
print(X_train_vec_seq[1])


-0.0383  0.0020 -0.0446  ...   0.0045  0.0211 -0.0224
 0.0175  0.0939 -0.0591  ...   0.0802 -0.0016 -0.0358
 0.0408  0.0206 -0.0234  ...   0.0331  0.0218 -0.0536
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
[torch.FloatTensor of size 20x400]



In [12]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [27]:
# Hyper Parameters
sequence_length = seq_len
input_size = 400
hidden_size = 256
num_layers = 1
num_classes = 2
batch_size = len(X_train_vec_seq)
num_epochs = 500
learning_rate = 0.01

In [28]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.log_softmax = nn.Softmax()
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        out = self.log_softmax(out)
        return out

rnn = RNN(input_size, hidden_size, num_layers, num_classes)

In [29]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(rnn.parameters(), lr = learning_rate)

In [16]:
#optimizer.lr = 0.1

In [30]:
# Train the Model
for epoch in range(num_epochs):
        tweet = Variable(X_train_vec_seq)
        label = Variable(y_train_torch.view(-1))
    
        optimizer.zero_grad()
        outputs = rnn(tweet)

        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print ('Epoch [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, loss.data[0]))
            
        if loss.data[0] < 0.001:
            print('done')
            break

Epoch [10/500], Loss: 0.6932
Epoch [20/500], Loss: 0.6932


KeyboardInterrupt: 

In [18]:
def score_model(model, X, y, model_name):
    from sklearn.metrics import roc_auc_score

    correct = 0
    total = 0
    positive = 0
    correct_positive = 0
    should_be_positive = 0

    tweet = Variable(X)
    label = y.view(-1)   
    output = model(tweet)
    a, predicted = torch.max(output.data, 1)

    total += label.size(0)
    correct += (predicted == label).sum()
    positive += predicted.sum()
    should_be_positive += label.sum()

    for i in range(len(predicted)):
        if predicted[i] == label[i] and label[i] == 1:
            correct_positive += 1


    precision = correct_positive / positive
    recall = correct_positive / should_be_positive
    f_score = 2 / ( (1 / precision) + (1 / recall) )


    print('%s Accuracy: %d %%' % (model_name, 100 * correct / total)) 
    print('%s F-score: %f' % (model_name, f_score)) 

    
    print('%s roc_auc_score: %f\n' % (model_name, roc_auc_score(label.numpy(), output.data.numpy()[:, 1])))
    


In [ ]:
score_model(rnn, X_train_vec_seq, y_train_torch, 'Train')
score_model(rnn, X_test_vec_seq, y_test_torch, 'test')